We will start by importing two useful packages: geopy and folium. Geopy provides tools for geocoding, and converting waypoints and bearings into other forms of useful information. Folium is a mapping package that allows us to plot our data on a map using leaflet.js. We will also import datetime to allow summation of time deltas.

In [1]:
%pip install geopy
%pip install folium
import folium
import geopy
from geopy.distance import geodesic
from datetime import datetime, timedelta
import folium.plugins as plugins

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Next, we set up an empty dictionary for waypoints. This is a list of dictionaries, each contianing a lat, long, and time. 
This cell is seperated out from the next set of code so it can be re-run, resetting the list to void. 

In [16]:
waypoints = []

To start filling the waypoint list, we ask the user for a starting waypoint. For the benifit of the package, and preventing bugs, we are assuming the starting time to be 1st Jan 1990 at midnight. 

In [17]:
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))

waypoints.append(
    {
        "latitude": latitude,
        "longitude": longitude,
        "dateTime": datetime.strptime("1800-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
    }
)

The next cell is re-runnable as often as required. It asks the user to provide a heading, speed, and time. It then uses geopy's geodesic function to calculate the distance travelled, and the new waypoint. This is then added to the list of waypoints.

In [20]:
bearing = float(input("Enter bearing: "))
speed = float(input("Enter speed: "))
time = float(input("Enter time in hours: "))

# get the previous waypoint
previous_latitude = waypoints[-1]["latitude"]
previous_longitude = waypoints[-1]["longitude"]
previous_time = waypoints[-1]["dateTime"]

# calculate the new waypoint
new_waypoint = geodesic(kilometers=speed * time).destination([
    previous_latitude,
    previous_longitude
], bearing)

waypoints.append(
    {
        "latitude": new_waypoint.latitude,
        "longitude": new_waypoint.longitude,
        "dateTime": previous_time + timedelta(hours=time)
    }
)

Offer the user a chance to print all the waypoints as a quick sanity check. 

In [21]:
for waypoint in waypoints:
    print(waypoint)

{'latitude': 0.0, 'longitude': 0.0, 'dateTime': datetime.datetime(1800, 1, 1, 0, 0)}
{'latitude': -0.7596699137352162, 'longitude': 0.0, 'dateTime': datetime.datetime(1800, 1, 1, 12, 0)}
{'latitude': -0.7595937713390581, 'longitude': 0.8085543391243254, 'dateTime': datetime.datetime(1800, 1, 1, 13, 0)}
{'latitude': 0.04225526169954722, 'longitude': 1.827871795048741, 'dateTime': datetime.datetime(1800, 1, 2, 1, 0)}


Next, we convert each pair of waypoints into a series of journey legs. This means we take waypont a and waypoint a+1, and combine them as required by the package. 

In [23]:
lines = []

for i in range(len(waypoints) - 1):
    lines.append(
        {
            "coordinates": [
                [waypoints[i]["latitude"], waypoints[i]["longitude"]],
                [waypoints[i + 1]["latitude"], waypoints[i + 1]["longitude"]]
            ],
            "dates": [
                str(waypoints[i]["dateTime"]),
                str(waypoints[i + 1]["dateTime"])
            ]
        }
    )
    

We print the lines dictionary as a sanity check. 

In [24]:
for line in lines:
    print(line)

{'coordinates': [[0.0, 0.0], [-0.7596699137352162, 0.0]], 'dates': ['1800-01-01 00:00:00', '1800-01-01 12:00:00']}
{'coordinates': [[-0.7596699137352162, 0.0], [-0.7595937713390581, 0.8085543391243254]], 'dates': ['1800-01-01 12:00:00', '1800-01-01 13:00:00']}
{'coordinates': [[-0.7595937713390581, 0.8085543391243254], [0.04225526169954722, 1.827871795048741]], 'dates': ['1800-01-01 13:00:00', '1800-01-02 01:00:00']}


Now we draw a world map again, but this time we're using the lines dictionary

In [25]:
lines_map = folium.Map(location=[0, 0], zoom_start=2)

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"]
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": "red"
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1h",
    add_last_point=True,
).add_to(lines_map)

lines_map